In [1]:
import pandas as pd

In [4]:
V = pd.read_parquet("../../results/clinvar/all.annot.parquet")
V

,chrom,pos,ref,alt,consequence
0,1,11114338,A,G,missense_variant
1,1,11124516,G,A,missense_variant
2,1,11124516,G,T,missense_variant
3,1,11128107,G,C,missense_variant
4,1,11128107,G,T,missense_variant
...,...,...,...,...,...
3112,X,154966634,G,A,stop_gained
3113,X,154969361,G,C,missense_variant
3114,X,154969417,G,A,missense_variant
3115,X,154984788,G,T,stop_gained


In [11]:
df = V.consequence.value_counts().to_frame().reset_index()
df

,consequence,count
0,stop_gained,1683
1,missense_variant,980
2,splice_donor_variant,173
3,splice_acceptor_variant,156
4,splice_region_variant,35
5,start_lost,33
6,splice_donor_5th_base_variant,21
7,splice_donor_region_variant,13
8,splice_polypyrimidine_tract_variant,8
9,synonymous_variant,5


In [14]:
df.consequence = r"\verb|" + df.consequence + r"|"
df

,consequence,count
0,\verb|stop_gained|,1683
1,\verb|missense_variant|,980
2,\verb|splice_donor_variant|,173
3,\verb|splice_acceptor_variant|,156
4,\verb|splice_region_variant|,35
5,\verb|start_lost|,33
6,\verb|splice_donor_5th_base_variant|,21
7,\verb|splice_donor_region_variant|,13
8,\verb|splice_polypyrimidine_tract_variant|,8
9,\verb|synonymous_variant|,5


In [16]:
print(df.to_latex(index=False))

\begin{tabular}{lr}
\toprule
consequence & count \\
\midrule
\verb|stop_gained| & 1683 \\
\verb|missense_variant| & 980 \\
\verb|splice_donor_variant| & 173 \\
\verb|splice_acceptor_variant| & 156 \\
\verb|splice_region_variant| & 35 \\
\verb|start_lost| & 33 \\
\verb|splice_donor_5th_base_variant| & 21 \\
\verb|splice_donor_region_variant| & 13 \\
\verb|splice_polypyrimidine_tract_variant| & 8 \\
\verb|synonymous_variant| & 5 \\
\verb|intron_variant| & 5 \\
\verb|stop_lost| & 3 \\
\verb|3_prime_UTR_variant| & 1 \\
\verb|upstream_gene_variant| & 1 \\
\bottomrule
\end{tabular}

